# План проекта

## 1. Инициализация Spark
- Создание SparkSession
- Настройка конфигурации приложения

## 2. Определение схем данных
- Продукты (product_id, product_name)
- Категории (category_id, category_name)
- Связи (product_id, category_id)

## 3. Создание тестовых данных
- Данные продуктов, категорий и связей

## 4. Реализация метода
- **Left join** products ↔ relations
- **Left join** результата ↔ categories
- Выбор нужных признаков (product_name, category_name)
- Сортировка

## 5. Визуализация
- Вывод исходных таблиц
- Все пары продукт-категория
- Продукты без категорий (NULL)
- Отсортированный вывод


In [ ]:
'''
Схема метода

Продукты         Связи            Категории
    │               │                 │
    └─── LEFT JOIN ─┘                 │
            │                         │
            └─────── LEFT JOIN ───────┘
                        │
                        ├─ SELECT (только названия продуктов)
                        │
                        └─ ORDER BY (сортируем)
                        '''

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Создаем Spark сессию
spark = SparkSession.builder \
    .appName("ProductCategoryPairs") \
    .getOrCreate()

# Метод с left join
def get_product_category_pairs(products_df, categories_df, relations_df):
    """
    Возвращает все пары "Имя продукта - Имя категории" и продукты без категорий

    Args:
        products_df: Датафрейм с продуктами (колонки 'product_id', 'product_name')
        categories_df: Датафрейм с категориями (колонки 'category_id', 'category_name')
        relations_df: Датафрейм со связями (колонки 'product_id', 'category_id')

    Returns:
        Датафрейм с колонками: product_name, category_name
    """
    # Создаем полное соединение продуктов и категорий через связи
    result_df = products_df \
        .join(relations_df, "product_id", "left") \
        .join(categories_df, "category_id", "left") \
        .select("product_name", "category_name") \
        .orderBy("product_name", "category_name")

    return result_df

In [2]:
# Создаем тестовые данные
# Схема для продуктов
products_schema = StructType([
    StructField("product_id", IntegerType(), True),
    StructField("product_name", StringType(), True)
])

# Схема для категорий
categories_schema = StructType([
    StructField("category_id", IntegerType(), True),
    StructField("category_name", StringType(), True)
])

# Схема для связей
relations_schema = StructType([
    StructField("product_id", IntegerType(), True),
    StructField("category_id", IntegerType(), True)
])

# Данные продуктов
products_data = [
    (1, "Ноутбук"),
    (2, "Мышь"),
    (3, "Клавиатура"),
    (4, "Наушники"),
    (5, "Монитор"),
    (6, "Телефон")  # Продукт без категорий
]

# Данные категорий
categories_data = [
    (1, "Электроника"),
    (2, "Компьютерная техника"),
    (3, "Периферия"),
    (4, "Аксессуары")
]

# Данные связей
relations_data = [
    (1, 1), (1, 2),  # Ноутбук: Электроника, Компьютерная техника
    (2, 3), (2, 4),  # Мышь: Периферия, Аксессуары
    (3, 3),          # Клавиатура: Периферия
    (4, 1), (4, 4),  # Наушники: Электроника, Аксессуары
    (5, 2)           # Монитор: Компьютерная техника
    # Телефон (id=6) без категорий
]

# Создаем датафреймы
products_df = spark.createDataFrame(products_data, products_schema)
categories_df = spark.createDataFrame(categories_data, categories_schema)
relations_df = spark.createDataFrame(relations_data, relations_schema)

# Вывод исходных данных
print("Продукты:")
products_df.show()

print("\nКатегории:")
categories_df.show()

print("\nСвязи:")
relations_df.show()

# Применение метода
result_df = get_product_category_pairs(products_df, categories_df, relations_df)

print("\nРезультат - все пары продукт-категория и продукты без категорий:")
result_df.orderBy("product_name", "category_name").show()



Продукты:
+----------+------------+
|product_id|product_name|
+----------+------------+
|         1|     Ноутбук|
|         2|        Мышь|
|         3|  Клавиатура|
|         4|    Наушники|
|         5|     Монитор|
|         6|     Телефон|
+----------+------------+


Категории:
+-----------+--------------------+
|category_id|       category_name|
+-----------+--------------------+
|          1|         Электроника|
|          2|Компьютерная техника|
|          3|           Периферия|
|          4|          Аксессуары|
+-----------+--------------------+


Связи:
+----------+-----------+
|product_id|category_id|
+----------+-----------+
|         1|          1|
|         1|          2|
|         2|          3|
|         2|          4|
|         3|          3|
|         4|          1|
|         4|          4|
|         5|          2|
+----------+-----------+


Результат - все пары продукт-категория и продукты без категорий:
+------------+--------------------+
|product_name|       cate